In [ ]:
import copy
import math
import functools

In [ ]:
import marimo as mo
import nbformat
import util

# `IBS_SCS` アルゴリズム

参考: Sayyed Rasoul Mousavi, Fateme Bahri, Farzaneh Sadat Tabataba, 2012, An enhanced beam search algorithm for the Shortest Common Supersequence Problem,
Engineering Applications of Artificial Intelligence,
Volume 25, Issue 3, Pages 457-467, https://www.sciencedirect.com/science/article/pii/S0952197611001497

- 計算量: $O(k^2 \log_2 q + L^* (n \kappa \beta q + \beta q \log_2 \left( \beta q \right)))$
    - (補足) $L^*$: このアルゴリズムによって返される解の長さ. $O(nk)$
    - (補足) $\kappa$, $\beta$: ビームサーチのパラメータ.
- 近似精度: ?

In [ ]:
@functools.cache
def probability(len_sub: int, len_super: int, num_alphabet: int) -> float:
    """
    一様ランダムに生成された文字列 w (長さ len_sub) と y (長さ len_super) について, 
    y が w の supersequence である確率を返す. 
    """

    if len_sub == 0:
        return 1.0
    elif len_sub > len_super:
        return 0.0
    else:
        tmp1 = 1.0 / num_alphabet * probability(len_sub - 1, len_super - 1, num_alphabet)
        tmp2 = (num_alphabet - 1) / num_alphabet * probability(len_sub, len_super - 1, num_alphabet)

    return tmp1 + tmp2

In [ ]:
class State:
    def __init__(self, instance: list[str]):
        self.instance = instance
        self.positions = [0 for _ in instance]
        self.solution = ""

    def is_feasibel(self) -> bool:
        return all(pos == len(s) for s, pos in zip(self.instance, self.positions))

    def is_usable(self, c: str) -> bool:
        for pos, s in zip(self.positions, self.instance):
            if pos < len(s) and s[pos] == c:
                return True

        return False

    def dominate(self, other: "State") -> bool:
        geq = all(pos1 >= pos2 for pos1, pos2 in zip(self.positions, other.positions))
        neq = any(pos1 != pos2 for pos1, pos2 in zip(self.positions, other.positions))
        return geq and neq

In [ ]:
def solve(instance: list[str], beta: int = 500, kappa: int = 10) -> str:
    chars = sorted(list(set("".join(instance))))
    initial_state = State(instance)
    b: list[State] = []

    while True:
        # Step 1: Extension
        c: list[State] = []
        for state in b:
            for char in chars:
                if not state.is_usable(char):
                    continue

                new_state = copy.deepcopy(state)
                new_state.solution += char
                for idx, (s, pos) in enumerate(zip(self.instance, new_state.positions)):
                    if pos < len(s) and s[pos] == char:
                        new_state.positions[idx] += 1

                if new_state.is_feasible(instance):
                    return new_state.solution
                else:
                    c.append(new_state)

        # Step 2: Evaluation of candidate solutions
        k = round(
            math.log2(len(chars))
            * max(
                len(s) - pos
                for state in c
                for s, pos in zip(instance, state.positions)
            )
        )

        heuristics = []
        for state in c:
            tmp_h = 1.0
            for s, pos in zip(instance, state.positions):
                tmp_h *= probability(len(s) - pos, k, len(chars))
            heuristics.append(tmp_h)

        # Step 3: Dominance pruning
        sorted_c = [
            state for (idx, state) in sorted(
                list(enumerate(c)),
                key=lambda tmp: heuristics[tmp[0]],
                reverse=True,
            )
        ]
        kappa_best_list = sorted_c[:kappa]
        for idx in range(len(sorted_c) - 1, len(kappa_best_list) - 1, -1):
            if any(better.dominate(sorted_c[idx]) for better in kappa_best_list):
                sorted_c.pop(idx)

        # Step 4: Selection
        b = sorted_c[:beta]

In [ ]:
_instance = util.parse("uniform_q26n004k015-025.txt")
util.show(_instance)
_solution = solve(_instance)
util.show(_instance, _solution)
print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf



<span class="codehilite"><div class="highlight"><pre><span></span><span class="gt">Traceback (most recent call last):</span>
  File <span class="nb">&quot;/home/psiana011/.cache/uv/archive-v0/UQfI4dWEzUU5zOQGwLwcg/lib/python3.13/site-packages/marimo/_runtime/executor.py&quot;</span>, line <span class="m">138</span>, in <span class="n">execute_cell</span>
<span class="w">    </span><span class="n">exec</span><span class="p">(</span><span class="n">cell</span><span class="o">.</span><span class="n">body</span><span class="p">,</span> <span class="n">glbls</span><span class="p">)</span>
<span class="w">    </span><span class="pm">~~~~^^^^^^^^^^^^^^^^^^</span>
  File <span class="nb">&quot;/tmp/nix-shell.luvIvd/marimo_1663786/__marimo__cell_CQMC_.py&quot;</span>, line <span class="m">3</span>, in <span class="n">&lt;module&gt;</span>
<span class="w">    </span><span class="n">_solution</span> <span class="o">=</span> <span class="n">solve</span><span class="p">(</span><span class="n">_inst

> /tmp/nix-shell.luvIvd/marimo_1663786/__marimo__cell_JXDq_.py(28)solve()
-> * max(
(Pdb) 

 bt


  /home/psiana011/.cache/uv/archive-v0/UQfI4dWEzUU5zOQGwLwcg/lib/python3.13/site-packages/marimo/_runtime/executor.py(138)execute_cell()
-> exec(cell.body, glbls)
  /tmp/nix-shell.luvIvd/marimo_1663786/__marimo__cell_CQMC_.py(3)<module>()
-> _solution = solve(_instance)
> /tmp/nix-shell.luvIvd/marimo_1663786/__marimo__cell_JXDq_.py(28)solve()
-> * max(
(Pdb) 

...

> /tmp/nix-shell.luvIvd/marimo_1663786/__marimo__cell_JXDq_.py(28)solve()
-> * max(
(Pdb) 

 s=


*** SyntaxError: invalid syntax
(Pdb) 

 print(s)


*** NameError: name 's' is not defined
(Pdb) 